# Project 3: Activity Detection with Machine Learning

In this project, we will use a database containing activity data collected in Project 2.
The main steps include:
 - **Loading** the database into a Pandas dataframe
 - **Extracting features** for machine learning
 - **Training** different models to find the best activity detection model

Your assignment is to complete the analysis and submit the required files.

In [5]:
# Mount Google Drive (Optional - for Colab users)
# from google.colab import drive
# drive.mount('/content/drive')

# Define the working directory (update this path if needed)
data_path = './'

## Submission Requirements
Your final submission should be a **single compressed file** containing:
 - **Completed Jupyter Notebook** (this file, with all outputs and analysis)
 - **PDF Report** with the required write-ups and explanations
 - **C Header File (`.h`)** containing the best-trained model ported to C

## Setp 1. Load the Activity Database
Ensure that the dataset file is located in the correct directory, or update the file path accordingly.

In [6]:
import pandas as pd
import os

# Define the path to the dataset (update if necessary)
db_path = os.path.join(data_path, 'dataset_spring_2024.csv') 

# Load the dataset into a Pandas dataframe
df = pd.read_csv(db_path, dtype={'label': 'str', 'participant': 'str'})

# Remove unnecessary columns (e.g., timestamp if not needed)
df = df.drop(columns=['time_ms'])

# Display basic dataset information
df.info()

# Display the first 10 rows to inspect the dataset
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768519 entries, 0 to 1768518
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   participant  object 
 1   accel_x      float64
 2   accel_y      float64
 3   accel_z      float64
 4   gyr_x        float64
 5   gyr_y        float64
 6   gyr_z        float64
 7   label        object 
dtypes: float64(6), object(2)
memory usage: 107.9+ MB


,participant,accel_x,accel_y,accel_z,gyr_x,gyr_y,gyr_z,label
0,P1,-0.10,-0.95,0.35,1.68,-3.08,-1.54,sitting
1,P1,-0.10,-0.94,0.36,2.03,-3.15,-1.47,sitting
2,P1,-0.10,-0.94,0.36,2.52,-3.22,-1.54,sitting
3,P1,-0.10,-0.94,0.36,2.52,-3.15,-1.61,sitting
4,P1,-0.10,-0.94,0.36,2.73,-3.01,-1.61,sitting
5,P1,-0.09,-0.94,0.37,2.87,-2.80,-1.82,sitting
6,P1,-0.09,-0.94,0.37,2.73,-2.45,-1.68,sitting
7,P1,-0.09,-0.94,0.37,2.80,-2.31,-1.75,sitting
8,P1,-0.08,-0.94,0.37,2.38,-2.17,-1.82,sitting
9,P1,-0.08,-0.94,0.37,1.82,-2.10,-1.75,sitting


In [7]:
# include the dataset from 2023
db_path = os.path.join(data_path, 'dataset_winter_2023.csv')
df_2023 = pd.read_csv(db_path)

# clean the data to match this year's dataset
df_2023['label'] = df_2023['label'].str.lower().str.strip()
df_2023['label'] = df_2023['label'].str.replace('sitting/working with a computer', 'sitting')
df_2023['label'] = df_2023['label'].str.replace('sitting/working\xa0with a computer', 'sitting')
print(df_2023.label.unique())

df_2023['participant'] = df_2023['participant'].astype(str)

# add 100 each participant id to separate
def _reformat_participant_id(id_str):
    id = int(id_str.lower().split('p')[-1])
    id += 100
    return f'P{id}'
df_2023['participant'] = df_2023['participant'].apply(_reformat_participant_id)

df_2023 = df_2023.drop(columns=['time'])

df_2023.head()

['sitting' 'standing' 'walking' 'ascending stairs' 'descending stairs'
 'jumping' 'running' 'dancing' 'rest']


,participant,accel_x,accel_y,accel_z,gyr_x,gyr_y,gyr_z,label
0,P119,-0.36,-0.94,-0.01,1.33,-1.75,0.35,sitting
1,P119,-0.36,-0.94,-0.01,1.47,-1.68,0.56,sitting
2,P119,-0.36,-0.94,-0.01,1.26,-1.89,0.14,sitting
3,P119,-0.36,-0.94,-0.01,1.05,-1.75,0.63,sitting
4,P119,-0.36,-0.94,-0.01,1.26,-1.89,0.42,sitting


In [8]:
# merge the database
df = pd.concat([df, df_2023], ignore_index=True)

# clean up
del df_2023

df.head()

,participant,accel_x,accel_y,accel_z,gyr_x,gyr_y,gyr_z,label
0,P1,-0.1,-0.95,0.35,1.68,-3.08,-1.54,sitting
1,P1,-0.1,-0.94,0.36,2.03,-3.15,-1.47,sitting
2,P1,-0.1,-0.94,0.36,2.52,-3.22,-1.54,sitting
3,P1,-0.1,-0.94,0.36,2.52,-3.15,-1.61,sitting
4,P1,-0.1,-0.94,0.36,2.73,-3.01,-1.61,sitting


## Setp 2. Extract Features

Here we will select a window size and compute aggregations on the dataset. The following line of code will create a new dataframe which will compute aggregate statistics across each participant, label, and window size.

**Note**: this will take some time to compute, especially on a Google Colab instance. The more complex the aggregation calculations, the longer it will take.


### <span style="color:red">Task 2.1</span>
<span style="color:red">Add at least 4 additional features to the feature set. We have provided `mean` and `std` (standard deviation)</span>

In [9]:
# Define window size for feature extraction
window_size = 100 # Corresponds to approximately 1 second of data

############### edit code below ############### 
df_grouped = df.groupby(['participant', 'label']).rolling(window=window_size).agg({
    # 'time_ms': ['sum'],
    'accel_x': ['mean', 'std', 'min', 'max', 'median', 'skew'],
    'accel_y': ['mean', 'std', 'min', 'max', 'median', 'skew'],
    'accel_z': ['mean', 'std', 'min', 'max', 'median', 'skew'],
    'gyr_x': ['mean', 'std', 'min', 'max', 'median', 'skew'],
    'gyr_y': ['mean', 'std', 'min', 'max', 'median', 'skew'],
    'gyr_z': ['mean', 'std', 'min', 'max', 'median', 'skew']
}).reset_index().dropna()

# flatten column names so there are is no column levels
df_grouped.columns = ['_'.join(col).strip() for col in df_grouped.columns.values]

# clean up columns
df_grouped.rename(columns={'participant_': 'participant', 'label_': 'label'}, inplace=True)
df_grouped.drop(columns=['level_2_'], inplace=True)

# # optional - save the database to avoid re-running the above code
# df_group_file_path = f'project2_class_dataset_grouped_w{window_size}.csv'
# df_grouped.to_csv(df_group_file_path, index=False)

# then read the csv file
# df_grouped = pd.read_csv(df_group_file_path)

df_grouped.head()

,participant,label,accel_x_mean,accel_x_std,accel_x_min,accel_x_max,accel_x_median,accel_x_skew,accel_y_mean,accel_y_std,...,gyr_y_min,gyr_y_max,gyr_y_median,gyr_y_skew,gyr_z_mean,gyr_z_std,gyr_z_min,gyr_z_max,gyr_z_median,gyr_z_skew
99,P1,ascending stairs,-0.8253,0.177869,-1.27,-0.53,-0.775,-0.718442,-0.5937,0.283623,...,-159.6,69.79,-43.715,-0.399055,-1.7871,40.846795,-102.06,53.48,16.59,-0.688015
100,P1,ascending stairs,-0.8254,0.177846,-1.27,-0.53,-0.780,-0.717088,-0.5904,0.282056,...,-159.6,69.79,-44.275,-0.369833,-1.2355,40.650189,-102.06,53.48,16.59,-0.728099
101,P1,ascending stairs,-0.8251,0.177880,-1.27,-0.53,-0.780,-0.721753,-0.5888,0.281939,...,-159.6,69.79,-44.695,-0.356676,-0.6027,40.319882,-102.06,53.48,16.59,-0.771554
102,P1,ascending stairs,-0.8249,0.177920,-1.27,-0.53,-0.775,-0.724572,-0.5875,0.282047,...,-159.6,69.79,-44.695,-0.363526,0.1239,39.762283,-102.06,53.48,16.59,-0.812405
103,P1,ascending stairs,-0.8242,0.177998,-1.27,-0.53,-0.770,-0.735353,-0.5869,0.282277,...,-159.6,69.79,-44.695,-0.368161,0.8106,39.170303,-102.06,53.48,16.59,-0.850316


## Setp 3. Feature Selection

Here we will find the *best* features to use in our model. This can be done by using a variety of techniques, including *forward selection*, *backward selection*, *L1 regularization*, or *Random Forest Importance*. See [feature selection in Python](https://scikit-learn.org/stable/modules/feature_selection.html) for more information.

We provide two methods for features selection, *Univariate Feature Selection* and *Recursive Feature Elimination (RFE)*.

In [11]:
X = df_grouped.drop(columns=['participant', 'label']) # all columns except grouped columns
y = df_grouped['label']

### 3.1 Univariate Feature Selection

### <span style="color:red">Task 3.1</span>
<span style="color:red">Explain the process of univariate feature selection (~1 paragraph, what is the purpose of this method? How does it work?). Which feature have the most importance according to this method?</span>

In [11]:
from sklearn.feature_selection import SelectKBest, f_classif

# Perform univariate feature selection
k = X.columns.size # select all features
selector = SelectKBest(f_classif, k=k)
X_new = selector.fit_transform(X, y)

features_selected = selector.get_support(indices=True)
selected_feature_names = X.columns[features_selected]

# print names and f_scores of selected features
selected_features = pd.DataFrame({'feature': selected_feature_names, 'f_score': selector.scores_[features_selected]})
selected_features = selected_features.sort_values(by='f_score', ascending=False)
selected_features.head(15)

,feature,f_score
1,accel_x_std,414149.491084
7,accel_y_std,274139.330284
13,accel_z_std,266514.200351
25,gyr_y_std,260368.015552
31,gyr_z_std,251370.765099
19,gyr_x_std,222869.245764
14,accel_z_min,175595.507974
26,gyr_y_min,174357.306896
33,gyr_z_max,171594.878481
27,gyr_y_max,165867.704992


### Task 3.1 Answer

Univariate feature selection is a statistical method used to evaluate each feature independently to determine its individual relationship with the target variable. The code above leverages SelectKBest with the ANOVA F-test (f_classif) to compute an F-score for every feature in the dataset, ranking them by their statistical significance. Essentially, the method tests each feature to see how well it distinguishes between different classes, with higher scores indicating a stronger discriminatory power. In this case, after fitting the selector on the features X and target y, the features are sorted by their F-scores, and the output shows that accel_x_std has the highest F-score (414149.491084), indicating that it is the most important feature according to this univariate selection approach.

### 3.2 Recursive Feature Elimination (RFE)

Here we use the method of RFE to select the best features. Different from the univariate feature selection, RFE selects features by recursively considering smaller and smaller sets of features. We choose how we evaluate the importance of a feature by setting the parameter `estimator`. In this case, we use a `RandomForestClassifier` to evaluate the importance of a feature.

**Note**: this will take some time to run. (ex: 5.5mins on M2 Macbook Pro)

### <span style="color:red">Task 3.2</span>
<span style="color:red">Explain the process of recursive feature elimination (~1 paragraph, what is the purpose of this method? How does it work?). Which feature have the most importance according to this method?</span>

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=5, # the number of trees in the forest
    random_state=42, # for reproducibility
    max_depth=5 # the maximum depth of the tree
)
rfe = RFE(estimator=model, n_features_to_select=4, step=1)

X_rfe = rfe.fit_transform(X, y)

selected_features_rfe = X.columns[rfe.support_]

print("Features selected by RFE:", selected_features_rfe)

Features selected by RFE: Index(['accel_x_std', 'accel_y_std', 'accel_z_mean', 'accel_z_median'], dtype='object')


### Task 3.2 Answer

Recursive Feature Elimination (RFE) is a backward selection technique used to identify the most relevant features by recursively fitting a model—in this case, a RandomForestClassifier—and eliminating the least important features at each step until the desired number of features remains. The process works by initially training the model on all features, ranking them based on their importance (as determined by the estimator), and then progressively removing the weakest feature one at a time(depending on the step parameter) before retraining the model. This iterative pruning helps in reducing the dimensionality of the dataset while retaining only those features that contribute most to the predictive power of the model. According to the provided code and its output, RFE selected the features `accel_x_std`, `accel_y_std`, `accel_z_mean`, and `accel_z_median`, with `accel_x_std` being the most important feature as it is typically ranked highest by the underlying estimator.

### <span style="color:red">Task 3.3: Additional Feature Selection Method</span>

<span style="color:red">Please implement an additional feature selection method of your choice. You can use any feature selection method from the [scikit-learn library](https://scikit-learn.org/stable/modules/feature_selection.html), or any others. In your report, writing a short description of the method and the results, similar to above but include **why** you selected this method.</span>

In [ ]:
## implement additional feature selection method here ...
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Initialize Logistic Regression with L1 penalty
model = LogisticRegression(penalty='l1', solver='saga', random_state=42, max_iter=10000, n_jobs=-1)
# Use SelectFromModel to automatically select features with non-zero coefficients
selector = SelectFromModel(model)
selector.fit_transform(X, y)

# Retrieve and print the selected feature names
selected_features_l1 = X.columns[selector.get_support()]
print("Features selected by SelectFromModel (L1):", selected_features_l1)


### Task 3.4: Feature Selection

Choose the best features from the methods above and create a new dataframe with only those features. In your report, note which features you selected, and why.

In [14]:
# example features, you should choose your own.
selected_features = ['accel_x_std', 'accel_y_std', 'accel_z_std', 'gyr_x_std', 'gyr_y_std', 'gyr_z_std']
df_selected_features = df_grouped[['participant', 'label'] + selected_features]

df_selected_features.head()

,participant,label,accel_x_std,accel_y_std,accel_z_std,gyr_x_std,gyr_y_std,gyr_z_std
99,P1,ascending stairs,0.177869,0.283623,0.234394,95.786530,41.125919,40.846795
100,P1,ascending stairs,0.177846,0.282056,0.233043,93.403866,41.128009,40.650189
101,P1,ascending stairs,0.177880,0.281939,0.231738,90.788858,41.195001,40.319882
102,P1,ascending stairs,0.177920,0.282047,0.229859,87.643445,41.065231,39.762283
103,P1,ascending stairs,0.177998,0.282277,0.229644,84.653801,40.958809,39.170303


## Setp 4. Model Training & Selection

### 4.1 Train/Test Split by Percentage

We divide our dataset into a training set and a test set. We will train the model on the training set and evaluate the model on the test set. We split with a percentage, where we use a randomly distribution of the percentage of the data for the training and testing - 33%.

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.33 # percentage of data for testing

X = df_selected_features[selected_features]
Y = df_selected_features['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### 4.2 Model Training

Here we will train a series of models and evaluate them. We have provided two models: `Decision Tree Classifier`, and `GradientBoostingClassifier`.

### <span style="color:red">Task 4.2</span>
<span style="color:red">Please train 4 additional classifiers model. For each, including the ones we include a brief description (~1 paragraph). For the models you choose, explain why you chose it. Please limit you selection to machine learning models, we will try out deep learning models in the next project. You are free to use any models from the [scikit-learn library](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning), you may also use other as long as they are not deep learning models (e.g., multi-layer neural networks).</span>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Train multiple models and compare performance
############### edit code below ############### 
models = []
models.append(DecisionTreeClassifier(random_state=0, max_depth=5))
models.append(GradientBoostingClassifier())

In [ ]:
results = []

############### edit code below ############### 
for model in models:
    print('training: ', model.__class__.__name__)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    print(' - accuracy: ', accuracy)
    results.append(
        {
            'name': model.__class__.__name__,
            'accuracy': sklearn.metrics.accuracy_score(y_test, y_pred),
            'confusion_matrix': sklearn.metrics.confusion_matrix(y_test, y_pred),
            'classification_report': sklearn.metrics.classification_report(y_test, y_pred)
        }
    )

### <span style="color:red">Task 4.3: Model Reporting</span>
<span style="color:red">Report the results of each model, which one performed the best? Please print the model results below, but include a short write-up in your report.</span>

In [ ]:
# model results
# ...

## 5. LOSO (Leave-One-Subject-Out) Experiment


We have XX participants in our dataset. In this experiment we will train our best model above, but instead of randomly splitting the data, we will use a Leave-One-Subject-Out (LOSO) method. This means we will train on all participants except one, and then evaluate the model on the left out participant. We will repeat this process for all participants and then average the results. We will then compare the results with the previous model evaluation.

### <span style="color:red">Task 5.1: LOSO</span>
<span style="color:red">Create the variables `X_test`, `y_test`, `X_train`, `y_train` for the LOSO experiment.</span>

In [ ]:
import random

num_participants = df_selected_features['participant'].unique().size
test_participant = df_selected_features['participant'].unique()[random.randint(0, num_participants)]

# implement code here to create X_train, y_train, X_test, y_test for LOSO
# ...

### <span style="color:red">Task 5.2: Train Model w/ LOSO</span>
<span style="color:red">Train the best model with the LOSO method. report the results, and compare with the previous model evaluation. Write a short explanation of your observations. Why do you think it perform better or worse?
</span>

In [ ]:
# train and report results
# ...

### <span style="color:red">Task 5.3 Cross Validation</span>
<span style="color:red">In the interest of time, we will not implement cross-validation. But please include a short explanation of how you would implement cross-validation, and why it is important. (~1 paragraph)</span>

## 6. Port Model

We will now port the model to C, for use in Arduino. Please include the C file in your submission, but we will not run it on the embedded device for this assignment. In the next assignment, we will run the model on the Arduino.

### 6.1 install `micromlgen`

[micromlgen](https://github.com/eloquentarduino/micromlgen) is a opensource project which will generate C code from your sklearn models

In [ ]:
pip install micromlgen

### <span style="color:red">Task 6.2: Port Best Model and save</span>

In [ ]:
from micromlgen import port

# Select the best performing model (e.g., Gradient Boosting)
c_code = port(models[0])

print(c_code)

# save/port the model to a directory
# ...

### <span style="color:red">Task 6.3: Model Review</span>
<span style="color:red">Take a look at the C code, what do you notice? Do you think the model will run on our microcontrollers? Why or why not? Please include a brief write-up in your report.</span>